In [1]:
from math import sin,cos,log,ceil
import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams ['font.family']='serif'
rcParams ['font.size']=16

In [2]:
g=9.8
v_t=4.9
C_D=1/5.0
C_L=1.0
v0=6.5
theta0=-0.1
x0=0.0
y0=25.0

In [3]:
def f(u):
    v=u[0]
    theta=u[1]
    x0=u[2]
    y0=u[3]
    return numpy.array([-g*sin(theta)-C_D/C_L*g/v_t**2*v**2,-g*cos(theta)/v+g/v_t**2*v,v*cos(theta),v*sin(theta)])

In [4]:
def euler_step(u,f,dt):
    return u+dt*f(u)

In [5]:
def get_diffgrid(u_current,u_fine,dt):
    N_current=len(u_current[:,0])
    N_fine=len(u_fine[:,0])
    grid_size_ratio=ceil(N_fine/N_current)
    diffgrid=dt*numpy.sum(numpy.abs(u_current[:,2]-u_fine[::grid_size_ratio,2]))
    return diffgrid

In [6]:
def rk2_step(u,f,dt):
    u_star=u+0.5*dt*f(u)
    return u+dt*f(u_star)

In [7]:
T=36.0
dt=0.01
N=int(T/dt)+1
u_euler=numpy.empty((N,4))
u_rk2=numpy.empty((N,4))
u_euler[0]=numpy.array([v0,theta0,x0,y0])
u_rk2[0]=numpy.array([v0,theta0,x0,y0])

In [8]:
import time

In [15]:
starttime1=time.clock()
for n in range(N-1):
    u_euler[n+1]=euler_step(u_euler[n],f,dt)
endtime1=time.clock()
print(endtime1-starttime1)

0.06868473051815016


In [16]:
starttime2=time.clock()
for n in range(N-1):
    u_rk2[n+1]=rk2_step(u_rk2[n],f,dt)
endtime2=time.clock()
print(endtime2-starttime2)

0.14699148153333041


In [ ]:
x_euler=u_euler[:,2]
y_euler=u_euler[:,3]
x_rk2=u_rk2[:,2]
y_rk2=u_rk2[:,3]

In [ ]:
idx_negative_euler=numpy.where(y_euler<0.0)[0]
if len(idx_negative_euler)==0:
    idx_ground_euler=N-1
    print('Euler intergration has not touched ground yet!')
else:
    idx_ground_euler=idx_negative_euler=idx_negative_euler[0]

idx_negative_rk2=numpy.where(y_rk2<0.0)[0]
if len(idx_negative_rk2)==0:
    idx_ground_rk2=N-1
    print('Runge-Kutta integration has not touched ground yet!')
else:
    idx_ground_rk2=idx_negative_rk2[0]

In [ ]:
print('Are the x-values close? {}'.format(numpy.allclose(x_euler,x_rk2)))
print('Are the y-values close? {}'.format(numpy.allclose(y_euler,y_rk2)))

In [ ]:
pyplot.figure(figsize=(10,6))
pyplot.subplot(121)
pyplot.grid(True)
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.plot(x_euler[:idx_ground_euler],y_euler[:idx_ground_euler],'k-',label='Euler')
pyplot.plot(x_rk2[:idx_ground_rk2],y_rk2[:idx_ground_rk2],'r--',label='RK2')
pyplot.title('distance traveled: {:.3f}'.format(x_rk2[idx_ground_rk2-1]))
pyplot.legend();

pyplot.subplot(122)
pyplot.grid(True)
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.plot(x_euler,y_euler,'k-',label='Euler')
pyplot.plot(x_rk2,y_rk2,'r--',label='RK2')
pyplot.xlim(0,5)
pyplot.ylim(1.8,2.5)

In [ ]:
dt_values=numpy.array([0.1,0.05,0.01,0.005,0.001])
u_values=numpy.empty_like(dt_values,dtype=numpy.ndarray)
for i,dt in enumerate(dt_values):
    N=int(T/dt)+1
    t=numpy.linspace(0.0,T,N)
    u=numpy.empty((N,4))
    u[0]=numpy.array([v0,theta0,x0,y0])
    for n in range(N-1):
        u[n+1]=rk2_step(u[n],f,dt)
    u_values[i]=u

In [ ]:
diffgrid=numpy.empty_like(dt_values)
for i,dt in enumerate(dt_values):
    diffgrid[i]=get_diffgrid(u_values[i],u_values[-1],dt)

In [ ]:
pyplot.figure(figsize=(6,6))
pyplot.grid(True)
pyplot.xlabel(r'$Delta t$',fontsize=18)
pyplot.ylabel(r'L_1$-norm of the grid differences',fontsize=18)
pyplot.xlim(1e-4,1)
pyplot.ylim(1e-4,1)
pyplot.axis('equal')
pyplot.loglog(dt_values[:-1],diffgrid[:-1],color='k',ls='--',lw=2,marker='o');

In [ ]:
r=2
h=0.001
dt_values=numpy.array([h,r*h,r**2*h])
u_values=numpy.empty_like(dt_values,dtype=numpy.ndarray)
for i,dt in enumerate(dt_values):
    N=int(T/dt)+1
    t=numpy.linspace(0.0,T,N)
    u=numpy.empty((N,4))
    u[0]=numpy.array([v0,theta0,x0,y0])
    for n in range(N-1):
        u[n+1]=rk2_step(u[n],f,dt)
    u_values[i]=u
alpha=(log(get_diffgrid(u_values[2],u_values[1],dt_values[2]))-log(get_diffgrid(u_values[1],u_values[0],dt_values[1])))/log(r)
print('The order of convergence is alpha = {:.3f}'.format(alpha))

In [ ]:
def leapfrog_step(unm1,u,f,dt):
    return unm1+2.0*dt*f(u)

In [ ]:
g=9.8
v_t=4.9
C_D=1/5.0
C_L=1.0
v0=6.5
theta0=-0.1
x0=0.0
y0=2.0
T=15.0
dt=0.01
N=int(T/dt)+1

In [ ]:
u_leapfrog=numpy.empty((N,4))
u_leapfrog[0]=numpy.array([v0,theta0,x0,y0])
u_leapfrog[1]=rk2_step(u_leapfrog[0],f,dt)

In [ ]:
for n in range(1,N-1):
    u_leapfrog[n+1]=leapfrog_step(u_leapfrog[n-1],u_leapfrog[n],f,dt)

In [ ]:
x_leapfrog=u_leapfrog[:,2]
y_leapfrog=u_leapfrog[:,3]
idx_negative_leapfrog=numpy.where(y_leapfrog<0.0)[0]
if len(idx_negative_leapfrog)==0:
    idx_negative_leapfrog=N-1
    print('The glider has not reached the ground yet!')
else:
    idx_ground_leapfrog=idx_negative_leapfrog[0]

In [ ]:
pyplot.figure(figsize=(11,8))
pyplot.subplot(121)
pyplot.grid(True)
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.plot(x_leapfrog[:idx_ground_leapfrog],y_leapfrog[:idx_ground_leapfrog],color='k',ls='-',lw=2)
pyplot.title('distance traveled : {:.3f}'.format(x_leapfrog[idx_ground_leapfrog-1]),fontsize=18);

pyplot.subplot(122)
pyplot.grid(True)
pyplot.xlabel('$x$')
pyplot.ylabel('$y$')
pyplot.plot(x_leapfrog[:idx_ground_leapfrog],y_leapfrog[:idx_ground_leapfrog],color='k',ls=':',lw=2)
pyplot.xlim(0,5)
pyplot.ylim(1.8,2.5)

In [ ]:
r=2
h=0.001
dt_values=numpy.array([h,r*h,r**2*h])
u_values=numpy.empty_like(dt_values,numpy.ndarray)
for i,dt in enumerate(dt_values):
    N=int(T/dt)+1
    t=numpy.linspace(0.0,T,N)
    u=numpy.empty((N,4))
    u[0]=numpy.array([v0,theta0,x0,y0])
    u[1]=rk2_step(u[0],f,dt)
    for n in range(1,N-1):
        u[n+1]=leapfrog_step(u[n-1],u[n],f,dt)
    u_values[i]=u
alpha=(log(get_diffgrid(u_values[2],u_values[1],dt_values[2]))-log(get_diffgrid(u_values[1],u_values[0],dt_values[1])))/log(r)
print('The order of convergence is alpha = {:.3f}'.format(alpha))